<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [15]:
import os
import requests
import pprint
import pandas as pd
import pandas_gbq
import numpy as np
import re
from time import sleep
from google.oauth2 import service_account
from bs4 import BeautifulSoup
from datetime import datetime



def set_pandas_gbq_credentials():
    credentials = service_account.Credentials.from_service_account_file(os.environ['GCP_SECRETPATH'] )
    pandas_gbq.context.credentials = credentials
    pandas_gbq.context.project = os.environ['GCP_PROJECID'] 
    
def scrape_url(url):
    sleep(0.5)
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        containers =  soup.find_all('div', {'class':"text-body editorial-html"})
        return containers[0].find_all('p')[0].text.replace(u'\xa0', u' ')  
    except:
        return ''
    
def request_api(seed=False):
    r = requests.get('https://polisen.se/api/events')
    df = pd.DataFrame(r.json())
    df['location_name'] = [x.get('name') for x in df['location']]
    df['gps'] = [x.get('gps') for x in df['location']]
    df['gps_lat'] = [float(x.split(',')[0]) for x in df['gps']]
    df['gps_lon'] = [float(x.split(',')[1]) for x in df['gps']]
    df['datetime'] = pd.to_datetime(df['datetime'],utc=True)
    df['date_requested'] = pd.to_datetime(datetime.today(), utc=True)
    df.drop(columns=['location'], inplace=True)
    if seed:
        df['details'] = [scrape_url(url) for url in df['url']]
    return df
    
def filter_newly_arrived(new_data, history, idx_cols=['id', 'location_name', 'datetime']):
    if len(new_data) >= 1:
        new_data = new_data.set_index(idx_cols)
        if len(history) >= 1:
            idx_history = history.groupby(idx_cols).count().index
        else:
            idx_history = []
        idx_add = set(new_data.index) - set(idx_history)
        if len(idx_add) >= 1:
            new_data = new_data.loc[list(idx_add)]
            return new_data.reset_index()


def update_table_raw(project_id='my-user-project-287504',
                     dataset_id='crime_statistics_polisenapi',
                     table_id='raw'):
    # Get history
    QUERY = f"""
            SELECT id, location_name, datetime, gps
            FROM `{project_id}.{dataset_id}.{table_id}`
            WHERE CAST(datetime as DATE) > DATE_SUB(CURRENT_DATE(), INTERVAL 60 DAY)
            """
    history = pandas_gbq.read_gbq(QUERY, project_id=project_id)
    # Get new data
    new_data = request_api()
    # Upload newly arrived data (id not exist in history)
    newly_arrived = filter_newly_arrived(new_data, history)
    # upload new data
    if newly_arrived is not None:
        pandas_gbq.to_gbq(newly_arrived, f'{dataset_id}.{table_id}', project_id=project_id, if_exists='append')
        print(f'{newly_arrived.shape[0]} rows added to table: {dataset_id}.{table_id}')
    else:
        print(f'0 rows added to table: {dataset_id}.{table_id}')
        
        
def upload_initial(df, project_id='my-user-project-287504',
                     dataset_id='crime_statistics_polisenapi',
                     table_id='raw'):
    pandas_gbq.to_gbq(df, f'{dataset_id}.{table_id}', project_id=project_id, if_exists='replace')




def main():
    project_id='my-user-project-287504'
    dataset_id = 'crime_statistics_polisenapi'
    raw_table_id = 'raw'
    set_pandas_gbq_credentials()
    return update_table_raw(project_id=project_id, dataset_id=dataset_id, table_id=raw_table_id)
# main()

In [2]:
!gcloud config get-value project

Your active configuration is: [ds-platform]
ds-smartsupply


Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update



In [6]:
!gcloud config set project my-user-project-287504

Updated property [core/project].


In [16]:
set_pandas_gbq_credentials()

In [6]:
from pprint import pprint


In [47]:
def scrape_url(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    containers =  soup.find_all('div', {'class':"text-body editorial-html"})
    return containers[0].find_all('p')[0].text.replace(u'\xa0', u' ')

In [8]:
def wikipedia_loc_list():
    url = 'https://sv.wikipedia.org/wiki/Lista_%C3%B6ver_stadsdelar_i_Uppsala#Kvarng%C3%A4rdet'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    toc =  soup.find_all('div', {'class':"toc", "id":'toc'})
    loc_list = []
    for x in toc[0].find_all('li',{'class':"toclevel-2"}):
        loc_list.append(x.find('span', {'class':"toctext"}).text)
    df_loc = pd.DataFrame({'district': loc_list[1:]})
    df_loc['city'] = 'Uppsala'
    df_loc['district'] = np.where( df_loc['district']=='Danmark-Säby[1]','Danmark-Säby', df_loc['district'] )
    set_pandas_gbq_credentials()
    upload_initial(df_loc, project_id='my-user-project-287504', 
                   dataset_id='crime_statistics_polisenapi',  
                   table_id='dim_district')
    return df_toc

In [2]:
from google.cloud import bigquery

bq_client = bigquery.Client()
bq_client

/opt/anaconda3/envs/dsdc-monitoring/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/anaconda3/envs/dsdc-monitoring/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see http

In [3]:
bq_client.project

'my-user-project-287504'

In [ ]:
# # set_pandas_gbq_credentials()
# upload_initial(df_loc, project_id='my-user-project-287504', 
#                dataset_id='crime_statistics_polisenapi',  
#                table_id='uppsala')

In [13]:
def extract_location_details(detail, district):

    detail = re.sub('[.,;!:]', ' ', str(detail))
    locwords = []
    loc_keywords = ['gata', 'vägen', 'torg', 'plan', 'leden', 'park']
    locwords.append(extract_keywords(detail, keywords = loc_keywords))
    for dist in district['district'].values:
        if dist.lower() in detail.lower():
            locwords.append(dist)
    if len(locwords)>=1:
        locword = ' '.join(np.unique(locwords))
    else:
        locword = ''
    return locword


def extract_keywords(detail, keywords):
    detail = re.sub('[.,;!:]', ' ', str(detail))
    keywords_out = []
    for keyword in keywords:
        for x in detail.lower().split(' '):
            if keyword in x:
                keywords_out.append(x)
    if len(keywords_out)>=1:
        keyword_out = ' '.join(np.unique(keywords_out))
    else:
        keyword_out = ''
    return keyword_out

def update_table_uppsala():
    district = pandas_gbq.read_gbq(
            f"""
            SELECT district
            FROM `my-user-project-287504.crime_statistics_polisenapi.dim_district`
            """, project_id='my-user-project-287504')

    raw = pandas_gbq.read_gbq("""
            SELECT *
            FROM `my-user-project-287504.crime_statistics_polisenapi.raw`
            WHERE CAST(datetime as DATE) > DATE_SUB(CURRENT_DATE(), INTERVAL 60 DAY)
            """
    )

    hist = pandas_gbq.read_gbq("""
            SELECT *
            FROM `my-user-project-287504.crime_statistics_polisenapi.uppsala`
            WHERE CAST(datetime as DATE) > DATE_SUB(CURRENT_DATE(), INTERVAL 60 DAY)
            """
    )
    raw = raw[raw['location_name'].str.lower()=='uppsala']
    newly_arrived = filter_newly_arrived(raw, hist)
    if newly_arrived is not None:
        details_list = newly_arrived['details']+' '+newly_arrived['summary']+' '+ newly_arrived['name']
        newly_arrived['location_details'] = [extract_location_details(detail) for detail in details_list.values]
        newly_arrived['location_details'] = newly_arrived['location_details'] +' ' +'Uppsala'
        pandas_gbq.to_gbq(newly_arrived, f'crime_statistics_polisenapi.uppsala', project_id='my-user-project-287504', if_exists='append')
        print(f'{newly_arrived.shape[0]} rows added to table: crime_statistics_polisenapi.uppsala')
    else:
        print(f'0 rows added to table: crime_statistics_polisenapi.uppsala')
# update_table_uppsala()

In [14]:
def seed_table_cities():
    newly_arrived = pandas_gbq.read_gbq("""
            SELECT *
            FROM `my-user-project-287504.crime_statistics_polisenapi.raw`
            --WHERE location_name in (select distinct city from 
            --`my-user-project-287504.crime_statistics_polisenapi.dim_district`)
            """
    )
    district = pandas_gbq.read_gbq(
            f"""
            SELECT district
            FROM `my-user-project-287504.crime_statistics_polisenapi.dim_district`
            """, project_id='my-user-project-287504')
    details_list = newly_arrived['details']+' '+newly_arrived['summary']+' '+ newly_arrived['name']
    newly_arrived['location_details'] = [extract_location_details(detail, district=district) for detail in details_list]
    newly_arrived['location_details'] = newly_arrived['location_details'] +' ' +newly_arrived['location_name']+' ' +'Sweden'

    upload_initial(newly_arrived, project_id='my-user-project-287504', 
                   dataset_id='crime_statistics_polisenapi',  
                   table_id='cities_refined')
from IPython import embed
seed_table_cities()

Downloading: 100%|██████████| 124/124 [00:01<00:00, 120.70rows/s]
1it [00:05,  5.10s/it]


In [ ]:
newly_arrived = pandas_gbq.read_gbq("""
            SELECT *
            FROM `my-user-project-287504.crime_statistics_polisenapi.cities_refined`
            """, project_id='my-user-project-287504'
    )


In [40]:
translator = Translator()
tr=translator.translate(df['details'][0], src='sv' , dest='en').text

TypeError: 'NoneType' object is not iterable

In [42]:
df['details'][0]

In [ ]:
batch_size = 20
n_batch = len(newly_arrived)//batch_size
dfs = np.array_split(newly_arrived, n_batch)

In [48]:
# from googletrans import Translator
# import random
# def merciful_translate(arr):
#     translated= []
#     ii = 0
#     for x in arr:
#         try:
#             rand = random.randint(1,10)
#             if ii==rand:
#                 t_sleep = random.randint(1,10)
#                 print(f'sleep {t_sleep} seconds')
#                 sleep(t_sleep)
#                 translator = Translator()
#             tr=translator.translate(x, src='sv' , dest='en').text
#             print(ii)
#             translated.append(tr)
#         except:
#             t_sleep = 1
#             print(f'interrupted, sleep {t_sleep} seconds')
#             sleep(t_sleep)
#             translator = Translator()
#             tr=translator.translate(x, src='sv' , dest='en').text
#             print(ii)
#             translated.append(tr)
#         ii+=1
#     return translated
    

# for df in dfs:
#     translator = Translator()
#     df['details'] = [translator.translate(x, src='sv' , dest='en').text if x is not None else 'None' for x in df['details'] ]
#     pandas_gbq.to_gbq(df, f'crime_statistics_polisenapi.cities_refined_en', 
#                       project_id='my-user-project-287504', if_exists='append')

#     sleep_time=120
#     print(f"sleeping {sleep_time}")
#     sleep(sleep_time)

1it [00:03,  3.03s/it]


sleeping 120


1it [00:04,  4.05s/it]


sleeping 120


1it [00:03,  3.73s/it]


...

1it [00:04,  4.17s/it]


sleeping 120


1it [00:05,  5.37s/it]


sleeping 120


AttributeError: 'NoneType' object has no attribute 'group'

,datetime,id,name,summary,type,url,location_name,gps,gps_lat,gps_lon,date_requested,details,location_details
126,2020-10-23 04:26:00+00:00,232227,"23 oktober 06:26, Rattfylleri, Täby",En bilist kör E18 norrut vid Lahäll och inring...,Rattfylleri,https://polisen.se/aktuellt/handelser/2020/okt...,Täby,"59.4419,18.07033",59.441900,18.070330,2020-10-23 07:29:22.894298+00:00,"Bilen framförs mitt i vägbanan, blinkar åt all...",Norr Täby Sweden
127,2020-10-22 16:59:00+00:00,232176,"22 oktober 18:59, Fylleri/LOB, Täby",Ordningsvakt omhändertog en berusad person på ...,Fylleri/LOB,https://polisen.se/aktuellt/handelser/2020/okt...,Täby,"59.4419,18.07033",59.441900,18.070330,2020-10-22 19:29:22.746144+00:00,Polisen övertog omhändertagandet och förde per...,kemistvägen Täby Sweden
128,2020-10-26 00:31:00+00:00,232621,"26 oktober 01:31, Rattfylleri, Täby",En polispatrull stoppar en personbil för kontr...,Rattfylleri,https://polisen.se/aktuellt/handelser/2020/okt...,Täby,"59.4419,18.07033",59.441900,18.070330,2020-10-26 07:33:03.277646+00:00,Vid kontroll av föraren visade denne tecken på...,Norr norrtäljevägen Täby Sweden
129,2020-10-26 17:53:00+00:00,232738,"26 oktober 18:53, Trafikkontroll, Umeå",Hastighetskontroll på Ersboda och i Bullmark.,Trafikkontroll,https://polisen.se/aktuellt/handelser/2020/okt...,Umeå,"63.825847,20.263035",63.825847,20.263035,2020-10-26 19:33:05.917686+00:00,Polisen har under kvällen haft hastighetskontr...,ersmarksvägen Umeå Sweden
130,2020-10-26 13:37:00+00:00,232704,"26 oktober 14:37, Rattfylleri, Umeå",Man misstänks för stöld och drograttfylleri.,Rattfylleri,https://polisen.se/aktuellt/handelser/2020/okt...,Umeå,"63.825847,20.263035",63.825847,20.263035,2020-10-26 19:33:05.917686+00:00,En patrull stoppar en bilist i höjd med Kronop...,kronoparken Umeå Sweden
131,2020-10-26 14:56:00+00:00,232723,"26 oktober 15:56, Stöld, Umeå",Unga män misstänks för stöld.,Stöld,https://polisen.se/aktuellt/handelser/2020/okt...,Umeå,"63.825847,20.263035",63.825847,20.263035,2020-10-26 19:33:05.917686+00:00,Polis beordras till Rådhustorget där en ordnin...,rådhustorget Umeå Sweden
132,2020-10-26 14:30:00+00:00,232693,"26 oktober 15:30, Trafikkontroll, Umeå",Hastighetskontroll på Östra Ersboda.,Trafikkontroll,https://polisen.se/aktuellt/handelser/2020/okt...,Umeå,"63.825847,20.263035",63.825847,20.263035,2020-10-26 19:33:05.917686+00:00,Under eftermiddagen har polisen haft en hastig...,kärnvägen Umeå Sweden
133,2020-10-27 17:55:00+00:00,232915,"27 oktober 18:55, Stöld, Umeå","Stöld i butik, Umeå centrum.",Stöld,https://polisen.se/aktuellt/handelser/2020/okt...,Umeå,"63.825847,20.263035",63.825847,20.263035,2020-10-27 19:35:23.923080+00:00,Polis beordras till en butik i Umeå centrum dä...,Centrum Umeå Sweden
134,2020-10-19 15:03:00+00:00,230642,"19 oktober 17:03, Trafikolycka, Umeå",Trafikolycka på Umedalen.,Trafikolycka,https://polisen.se/aktuellt/handelser/2020/okt...,Umeå,"63.825847,20.263035",63.825847,20.263035,2020-10-19 22:07:39.847756+00:00,Tidigare i dag inträffade en singelolycka på J...,vägen Umeå Sweden
135,2020-10-19 10:59:00+00:00,230606,"19 oktober 12:59, Trafikolycka, Umeå",Trafikolycka på E4 Sävar.,Trafikolycka,https://polisen.se/aktuellt/handelser/2020/okt...,Umeå,"63.825847,20.263035",63.825847,20.263035,2020-10-19 22:07:39.847756+00:00,Polisen har kontaktats angående en personbil s...,Norr Umeå Sweden


In [44]:

def seed_table_cities_en():
    newly_arrived = pandas_gbq.read_gbq("""
            SELECT *
            FROM `my-user-project-287504.crime_statistics_polisenapi.cities_refined`
            """
    )
    
    newly_arrived['details'] = merciful_translate(newly_arrived['details'])
    upload_initial(newly_arrived, project_id='my-user-project-287504', 
                   dataset_id='crime_statistics_polisenapi',  
                   table_id='cities_refined_en')
from IPython import embed
seed_table_cities_en()

Downloading: 100%|██████████| 1770/1770 [00:01<00:00, 981.33rows/s]


0
interrupted, sleep 300 seconds
1
interrupted, sleep 300 seconds
2
3
4
5
6
7
8
9
10
11
12
13
14
15
interrupted, sleep 300 seconds
16
17
interrupted, sleep 300 seconds
18
19
20
interrupted, sleep 300 seconds
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
sleep 5 seconds
44
45
46
47
48
sleep 10 seconds
49
interrupted, sleep 300 seconds
50
51
52
interrupted, sleep 300 seconds
53
interrupted, sleep 300 seconds
54
55
56
interrupted, sleep 300 seconds
57
58
interrupted, sleep 300 seconds
59
interrupted, sleep 300 seconds
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
interrupted, sleep 300 seconds
84
85
interrupted, sleep 300 seconds
86
87
interrupted, sleep 300 seconds
88
89
90
91
92
93
94
95
96
97
98
interrupted, sleep 300 seconds
99
interrupted, sleep 300 seconds
100
interrupted, sleep 300 seconds
101
102
103
104
105
106
107
108
109
110
111
112
interrupted, sleep 300 seconds
113
114
115
116
117
118
119
120
121
122
interrupted, sleep 300 seco

AttributeError: 'NoneType' object has no attribute 'group'

In [22]:
translator = Translator()
translator.translate('jag heter uwe', src='sv' , dest='en').text

'my name is uwe'

In [9]:
data_source_query = """
(SELECT id, Datetime(datetime, 'Europe/Stockholm') datetime, location_name city, 
type, summary, details, concat(name,' ',summary, ' ',details  ) location_details, 
case when details like '%skottloss%' or details like '%pistol%' then 'gun-related' else 'other' end as gun_filter
FROM `my-user-project-287504.crime_statistics_polisenapi.raw`
WHERE location_name not in (
  SELECT DISTINCT city FROM `my-user-project-287504.crime_statistics_polisenapi.dim_district`)
order by datetime desc)
UNION ALL
(SELECT id, Datetime(datetime, 'Europe/Stockholm') datetime, location_name city, 
type, summary, details, location_details,
case when details like '%skottloss%' or details like '%pistol%' then 'gun-related' else 'other' end as gun_filter
FROM `my-user-project-287504.crime_statistics_polisenapi.cities_refined`
order by datetime desc)
"""

In [115]:


raw = raw[raw['location_name'].str.lower()=='uppsala']
details_list = raw['details']+' '+raw['summary']+' '+ raw['name']
raw['location_details'] = [extract_location_details(detail) for detail in details_list]

        
upload_initial(raw, project_id='my-user-project-287504', 
               dataset_id='crime_statistics_polisenapi',  
               table_id='uppsala')

1it [00:06,  6.77s/it]


In [119]:
np.sum(raw['location_name'].str.lower()=='uppsala')

20

In [39]:
district['city']='Uppsala'
upload_initial(district, project_id='my-user-project-287504', 
               dataset_id='crime_statistics_polisenapi',  
               table_id='dim_district')

1it [00:03,  3.42s/it]


In [88]:
import re
re.sub('[.,;!:]', ' ', detail)

'Två personer som har relation har ett ljudligt gräl med varandra på en adress i Sala backe och flera personer ringer 112 När polisen kommer till platsen är ingen skadad  men det har brunnit i en säng och polisen kommer att skriva en anmälan om mordbrand alternativt försök till mordbrand  En av personerna omhändertas med stöd i lagen om psykiatrisk tvångsvård '

In [8]:
def city_district(cityname, district_list):
    df_district = pd.DataFrame({'district': district_list})
    df_district['city'] = cityname
    return df_district

def get_dim_district():
    uppsala_list = ['Fjärdingen', 'Berthåga', 'Husbyborg', 'Hällby', 'Librobäck',
       'Luthagen', 'Rickomberga', 'Stenhagen', 'Eriksberg', 'Flogsta',
       'Ekeby', 'Håga', 'Kvarnbo', 'Kåbo', 'Norby', 'Polacksbacken',
       'Starbo', 'Gottsunda', 'Sunnersta', 'Ulleråker', 'Ultuna',
       'Valsätra', 'Vårdsätra', 'Bergsbrunna', 'Danmark-Säby', 'Nåntuna',
       'Sävja', 'Vilan', 'Boländerna', 'Fyrislund', 'Fålhagen',
       'Kungsängen', 'Kuggebro', 'Sala backe', 'Slavsta', 'Vaksala',
       'Årsta', 'Brillinge', 'Gamla Uppsala', 'Gränby', 'Kvarngärdet',
       'Löten', 'Nyby', 'Svartbäcken', 'Tunabackar', 'Ärna', 'Storvreta',
        'Rasbo','Centrum', 'Skuttunge', 'Skyttorp','Tycho Hedéns väg']

    stockholm_list = ['Bromma','Enskede','Årsta','Vantörs','Farsta','Hägersten','Älvsjö','Hässelby','Vällingby','Kungsholmens',
    'Norrmalms','Rinkeby','Kista','Skarpnäcks','Skärholmens','Spånga','Tensta','Södermalms','Östermalms', 'Täby', 'Solna', 'Sundbyberg']

    gavle_list = ['Alderholmen','Andersberg','Bomhus','Brynäs','Fredriksskans','Fridhem','Järvsta','Gamla Gävle',
             'Hagaström','Hemlingby','Hemsta','Hille','Varva','Höjersdal','Lexe','Nordost','Norr','Norrtull',
             'Nynäs','Näringen','Olsbacka','Stigslund','Strömsbro','Sätra','Söder',
             'Södertull','Sörby','Sörby urfjäll','Vall','Vallbacken','Villastaden','Väster','Tolvfors','Åbyggeby','Öster']
    karlskrona_list = ['Aspö','Augerum','Flymen','Fridlevstad','Hasslö','Jämjö','Karlskrona','Kristianopel','Lösen',
                       'Nättraby','Ramdala','Rödeby','Sillhövda','Sturkö','Torhamn','Tving']
    
    dfs = [city_district('Uppsala', uppsala_list),
           city_district('Stockholm', stockholm_list),
           city_district('Gävle', gavle_list),
           city_district('Karlskrona', karlskrona_list)
          ]
    return pd.concat(dfs)
dim_district = get_dim_district()
upload_initial(dim_district, project_id='my-user-project-287504', 
               dataset_id='crime_statistics_polisenapi',  
               table_id='dim_district')

1it [00:04,  4.27s/it]
